In [ ]:
import os
from ultralytics import YOLO
import yaml
from pathlib import Path
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image
import numpy as np
from source.visual_genome_meta_data import read_json_to_dict
from source.visual_genome_meta_data import get_image_meta_data
from source.visual_genome_data import count_occurrences
from source.visual_genome_to_yolo import create_class_mapping_from_list
from source.visual_genome_to_yolo import save_class_map_to_yaml
from source.visual_genome_to_yolo import convert_single_image_to_yolo
from source.visual_genome_to_yolo import read_yaml_to_class_map
from source.visual_genome_to_yolo import read_yolo_metadata
from source.visual_genome_to_yolo import visual_genome_to_yolo_data_n
from source.visual_genome_meta_data import plot_image_with_multiple_bboxes
from source.visual_genome_data import get_image_ids

In [ ]:
from PIL import Image

In [ ]:
def train_yolo_on_single_class(
    data_dir,          # Directory with your dataset
    class_yaml_path,   # Path to your class_map.yaml file
    target_class,      # The single class to train on (e.g., 'mountain')
    output_dir,        # Directory to save results
    epochs=50,         # Number of training epochs
    img_size=640,      # Input image size
    batch_size=16,     # Batch size
    device=None        # Device for training - will auto-select MPS if available
):
    """
    Train YOLO v11 on a single class from your dataset using M1 Mac's MPS acceleration.
    """
    import os
    import yaml
    import torch
    from ultralytics import YOLO  # Assuming this is the import for YOLOv11
    
    # Auto-select the best available device with priority for MPS on Mac
    if device is None:
        if torch.backends.mps.is_available():
            device = 'mps'
            print(f"Using MPS acceleration on Apple Silicon")
        elif torch.cuda.is_available():
            device = 'cuda'
            print(f"Using CUDA acceleration")
        else:
            device = 'cpu'
            print(f"Using CPU for training (this will be slow)")
    
    # Ensure output directory exists
    os.makedirs(output_dir, exist_ok=True)
    
    # 1. Load the full class map
    with open(class_yaml_path, 'r') as f:
        class_data = yaml.safe_load(f)
    
    # Find the index of the target class
    all_classes = class_data['names']
    if target_class not in all_classes:
        raise ValueError(f"Class '{target_class}' not found in class map. Available classes: {all_classes}")
    
    target_class_id = all_classes.index(target_class)
    print(f"Training on class: {target_class} (class_id: {target_class_id})")
    
    # 2. Create a new YAML file for single-class training
    single_class_yaml = os.path.join(output_dir, f"single_class_{target_class}.yaml")
    
    # Update dataset configuration
    train_path = os.path.join(data_dir, 'train/images')
    val_path = os.path.join(data_dir, 'val/images')
    
    single_class_config = {
        'path': os.path.abspath(data_dir),
        'train': 'train/images',
        'val': 'val/images',
        'nc': 1,  # Just one class
        'names': [target_class]  # Only the target class
    }
    
    with open(single_class_yaml, 'w') as f:
        yaml.dump(single_class_config, f)
    
    print(f"Created single-class configuration at: {single_class_yaml}")
    
    # 3. Initialize YOLO model
    model = YOLO("yolo11n.pt")  # Using YOLOv11 nano (smallest model)
    
    # 4. Train the model with MPS-specific configurations
    results = model.train(
        data=single_class_yaml,
        epochs=epochs,
        imgsz=img_size,
        batch=batch_size,
        project=output_dir,
        name=f"yolov11_{target_class}",
        exist_ok=True,
        patience=0,  # Early stopping
        verbose=True,
        device=device,  # Specify device for training
        amp=True       # Enable mixed precision training (improves performance on M1/M2)
    )
    
    print(f"Training completed. Model saved to {output_dir}/yolov11_{target_class}")
    return results

In [ ]:
#model = YOLO('yolov8n.pt')

In [ ]:
def test_and_plot_yolo_model(model_path, test_images_dir, output_dir=None, conf_threshold=0.3):
   """
   Test YOLO model and plot results with bounding boxes.
   Combines testing metrics with visual plotting.
   """
   import os
   import glob
   import matplotlib.pyplot as plt
   import matplotlib.patches as patches
   import numpy as np
   from PIL import Image
   from ultralytics import YOLO
   from pathlib import Path
   
   # Load model
   model = YOLO(model_path)
   
   # Get test images
   test_images = [f for f in os.listdir(test_images_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png', '.tif'))]
   
   # Create output directory if specified
   if output_dir:
       os.makedirs(output_dir, exist_ok=True)
   
   # Summary statistics
   total_images = len(test_images)
   images_with_detections = 0
   total_detections = 0
   
   # Process each image
   for i, img_file in enumerate(test_images):
       img_path = os.path.join(test_images_dir, img_file)
       
       # Run inference
       results = model(img_path, conf=conf_threshold)
       detections = results[0].boxes
       
       # Count detections
       if len(detections) > 0:
           images_with_detections += 1
           total_detections += len(detections)
       
       # Plot every 5th image or save all if output_dir specified
       should_plot = (i % 5 == 0) or output_dir
       
       if should_plot:
           img = Image.open(img_path)
           fig, ax = plt.subplots(figsize=(10, 10))
           ax.imshow(np.array(img))
           
           # Add bounding boxes
           for det in detections:
               box = det.xyxy[0].cpu().numpy()
               rect = patches.Rectangle(
                   (box[0], box[1]), box[2] - box[0], box[3] - box[1],
                   linewidth=2, edgecolor='red', facecolor='none'
               )
               ax.add_patch(rect)
               
               cls_id = int(det.cls[0])
               conf = float(det.conf[0])
               class_name = model.names[cls_id]
               ax.text(box[0], box[1] - 5, f"{class_name}: {conf:.2f}", 
                      color='red', fontsize=12, bbox=dict(facecolor='white', alpha=0.7))
           
           ax.set_title(f"Image {i+1}/{total_images}: {len(detections)} detections")
           plt.axis('off')
           plt.tight_layout()
           
           # Save or show
           if output_dir:
               plt.savefig(os.path.join(output_dir, f"result_{Path(img_file).stem}.jpg"), 
                          bbox_inches='tight', dpi=150)
               plt.close()
           else:
               plt.show()
               plt.close()
       
       if (i+1) % 10 == 0:
           print(f"Processed {i+1}/{total_images} images")
   
   # Print summary
   print(f"\nTest Results Summary:")
   print(f"Total images: {total_images}")
   print(f"Images with detections: {images_with_detections} ({images_with_detections/total_images*100:.1f}%)")
   print(f"Total detections: {total_detections}")
   print(f"Average detections per image: {total_detections/total_images:.2f}")
   
   return results

In [ ]:
root_path = Path('/Users/stephanehess/Documents/CAS_AML/dias_digit_project/test_yolo_object_train')
data_dir = root_path / 'yolo_object_train'
class_yaml_path = data_dir / 'class_map.yaml'

target_class = 'lighthouse'
output_dir_after = data_dir / 'output_test_posttrain'
output_dir_train = data_dir / 'output_train'
output_dir_test_before = data_dir / 'output_test_pretrain'



In [ ]:
print(root_path)
print(data_dir)
print(class_yaml_path)
print(output_dir_test_before)

In [ ]:
#model_path = data_dir / 'output/yolov11_mountain/weights/best.pt'
#model_path = data_dir / 'output/yolov11_church/weights/best.pt'
#model_path = data_dir / 'output/yolov11_church/weights/last.pt'


In [ ]:
test_images_dir = data_dir / 'test'
test_images_dir

### Initialize YOLO model: 

In [ ]:
# Initialize YOLO model
model = YOLO("yolo11n.pt")  # Using YOLOv11 nano (smallest model)

### Test untrained model:

In [ ]:
# Step 1: Test untrained model
print("=== Testing Untrained Model ===")
test_and_plot_yolo_model(
    model_path="yolo11n.pt",
    test_images_dir=test_images_dir,
    output_dir=output_dir_test_before,
    conf_threshold=0.1
)

### Train model to recognise lighthouses: 

In [ ]:
# Step 2: Train model  
print("\n=== Training Model ===")
training_results = train_yolo_on_single_class(
    data_dir=data_dir,
    class_yaml_path=class_yaml_path,
    target_class="lighthouse",
    output_dir=output_dir_train
)

In [ ]:
model_path = output_dir_train / "yolov11_lighthouse/weights/best.pt"

In [ ]:
# Step 3: Test trained model
print("\n=== Testing Trained Model ===")
test_and_plot_yolo_model(
    model_path=model_path,
    test_images_dir=test_images_dir, 
    output_dir=output_dir_after,
    conf_threshold=0.1
)


In [ ]:
output_dir_train

In [ ]:
model_path

In [ ]:
np.log(1)

In [ ]:
np.log(0)